In [1]:
import struct
import gzip
from fastcore.xtras import Path
from io import BufferedReader, BytesIO
from teleparser.main import BufferManager

folder = Path("/home/rsilva/volte_claro/")
folder.ls()

(#215) [Path('/home/rsilva/volte_claro/DIAMETERACCT_20250408_113036_02B3S.gz'),Path('/home/rsilva/volte_claro/DIAMETERACCT_20250408_100207_0287S.gz'),Path('/home/rsilva/volte_claro/DIAMETERACCT_20250408_101352_028CS.gz'),Path('/home/rsilva/volte_claro/DIAMETERACCT_20250408_124315_02D6S.gz'),Path('/home/rsilva/volte_claro/DIAMETERACCT_20250408_111836_02ADS.gz'),Path('/home/rsilva/volte_claro/DIAMETERACCT_20250408_100812_028AS.gz'),Path('/home/rsilva/volte_claro/DIAMETERACCT_20250408_121247_02C7S.gz'),Path('/home/rsilva/volte_claro/DIAMETERACCT_20250408_110016_02A3S.gz'),Path('/home/rsilva/volte_claro/DIAMETERACCT_20250408_104230_029BS.gz'),Path('/home/rsilva/volte_claro/DIAMETERACCT_20250408_104030_029AS.gz'),Path('/home/rsilva/volte_claro/DIAMETERACCT_20250408_111627_02ABS.gz'),Path('/home/rsilva/volte_claro/DIAMETERACCT_20250408_112436_02B0S.gz'),Path('/home/rsilva/volte_claro/DIAMETERACCT_20250408_120040_02C2S.gz'),Path('/home/rsilva/volte_claro/DIAMETERACCT_20250408_125659_02DES.gz'

In [ ]:
class CommandFlags:
    """Command Flags parser for Diameter protocol messages

    Parses an 8-bit byte according to the following rules:
    Bit 0: R - Set to 1 if ACR message, 0 if ACA message
    Bit 1: P - Set to 1 if message is proxiable (always 1 for ACR/ACA)
    Bit 2: E - Set to 1 if ACA contains Protocol Error 3xxx, 0 otherwise
    Bit 3: T - Set to 1 if retransmitted message, 0 otherwise
    Bits 4-7: Reserved (must be 0)
    """

    def __init__(self, flags_byte: int):
        """Initialize with a single byte (0-255) or hex value"""
        if not (0 <= flags_byte <= 255):
            raise ValueError(
                f"Command flags must be a single byte (0-255), got: {flags_byte}"
            )

        self.flags_byte = flags_byte
        self._parse_flags()

    def _parse_flags(self):
        """Parse individual flag bits from the byte"""
        # Extract individual bits (bit 0 is MSB, bit 7 is LSB)
        self.R = bool(self.flags_byte & 0x80)  # Bit 0 (MSB)
        self.P = bool(self.flags_byte & 0x40)  # Bit 1
        self.E = bool(self.flags_byte & 0x20)  # Bit 2
        self.T = bool(self.flags_byte & 0x10)  # Bit 3

        if self.E and self.R:
            raise ValueError(
                "Protocol Error 3xxx flag (E=1) is only valid for ACA messages, got R=1 which marks an ACR message"
            )

        # Check reserved bits (bits 4-7) should be 0
        reserved_bits = self.flags_byte & 0x0F
        if reserved_bits != 0:
            raise ValueError(f"Reserved bits 4-7 must be 0, got: {bin(reserved_bits)}")

    @property
    def is_acr_message(self) -> bool:
        """Returns True if this is an ACR message (R=1), False if ACA (R=0)"""
        return self.R

    @property
    def is_aca_message(self) -> bool:
        """Returns True if this is an ACA message (R=0, P=1), False if ACR (R=1)"""
        return not self.R and self.P

    @property
    def is_proxiable(self) -> bool:
        """Returns True if message is proxiable (should always be True for ACR/ACA)"""
        return self.P

    @property
    def has_protocol_error(self) -> bool:
        """Returns True if ACA contains Protocol Error 3xxx"""
        return self.E

    @property
    def is_retransmitted(self) -> bool:
        """Returns True if this is a retransmitted message"""
        return self.T

    @property
    def value(self) -> dict:
        """Returns a dictionary with all flag values"""
        return {
            "R": self.R,
            "P": self.P,
            "E": self.E,
            "T": self.T,
            "isACR": self.is_acr_message,
            "isACA": self.is_aca_message
        }

    def __str__(self) -> str:
        """String representation showing flag states"""
        msg_type = "ACR" if self.is_acr_message else "ACA"
        flags = []
        if self.P:
            flags.append("Proxiable")
        if self.E:
            flags.append("Protocol Error")
        if self.T:
            flags.append("Retransmitted")

        flag_str = ", ".join(flags) if flags else "None"
        return f"{msg_type} message - Flags: {flag_str}"

    def __repr__(self) -> str:
        return f"CommandFlags(0x{self.flags_byte:02X}) - R:{int(self.R)} P:{int(self.P)} E:{int(self.E)} T:{int(self.T)}"


def parse_command_flags(flags_byte: int) -> CommandFlags:
    """Parse command flags from a single byte

    Args:
        flags_byte: Integer value (0-255) representing the 8-bit command flags

    Returns:
        CommandFlags object with parsed flag values

    Example:
        >>> flags = parse_command_flags(0xC0)  # R=1, P=1, E=0, T=0
        >>> print(flags.is_acr_message)  # True
        >>> print(flags.is_proxiable)    # True
        >>> print(flags)                 # ACR message - Flags: Proxiable
    """
    return CommandFlags(flags_byte)


In [2]:
file = folder.ls().shuffle()[0]
file_handle = gzip.open(file, "rb")
bdata = file_handle.read(24)
file_handle.close()

In [3]:
bdata[:1000]

b'\xf4\x04\x01\x00\x04\xf4\xd0\x00\x01\x0f\x00\x00\x00\x03<\xd6\xf8@\x05Qx\xb6\x00\x00'

In [4]:
int.from_bytes(bdata[:2])

62468

In [22]:
(version, msg_len_bytes, flags, cmd_bytes, app_id, hbh_id, e2e_id) = struct.unpack(
    ">B3sB3sIII", bdata[2:22]
)


In [23]:
version

1

In [24]:
int.from_bytes(msg_len_bytes)

1268

In [25]:
str(parse_command_flags(flags))

'ACR message - Flags: Proxiable, Retransmitted'

In [19]:
flags.value

{'R': True,
 'P': True,
 'E': False,
 'T': True,
 'isACR': True,
 'isACA': False,
 'isProxiable': True,
 'hasProtocolError': False,
 'isRetransmitted': True}

In [9]:
int.from_bytes(cmd_bytes)

271

In [10]:
app_id

3

In [11]:
hbh_id

1020721216

In [12]:
e2e_id

89225398